# Hello World

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'your_open_key'

In [2]:
from swarm import Swarm, Agent

client = Swarm()

def transfer_to_agent_b():
    print("Transferring to Agent B...")
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Haikus.",
)

response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])

Transferring to Agent B...
Agent B not here,  
I will help with what you need,  
Ask your question, friend.


In [3]:
import json

from swarm import Agent


def get_weather(location, time="now"):
    """Get the current weather in a given location. Location MUST be a city."""
    return json.dumps({"location": location, "temperature": "65", "time": time})


def send_email(recipient, subject, body):
    print("Sending email...")
    print(f"To: {recipient}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    return "Sent!"


weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather, send_email],
)

In [4]:

response = client.run(
    agent=weather_agent,
    messages=[{"role": "user", "content": "What's the weather in NYC?"}],
)


In [5]:
print(response.messages[-1]["content"])

The current temperature in New York City is 65°F.


In [6]:
response = client.run(
    agent=weather_agent,
    messages=[{"role": "user", "content": "Can you send an email to sinan@loopgenius.com with a joke?. Make the subject a haiku."}],
)

Sending email...
To: sinan@loopgenius.com
Subject: Laughter in the air,
Smiles spreading from ear to ear.
Joy, our hearts declare.
Body: Why don't scientists trust atoms? 
Because they make up everything!


In [7]:
print(response.messages[-1]["content"])

I've sent the email with the subject in a haiku form and a joke for Sinan.


In [8]:
from swarm import Swarm
import pytest

client = Swarm()


def run_and_get_tool_calls(agent, query):
    message = {"role": "user", "content": query}
    response = client.run(
        agent=agent,
        messages=[message],
        execute_tools=False,
    )
    return response.messages[-1].get("tool_calls")


def test_calls_weather_when_asked(query):
    tool_calls = run_and_get_tool_calls(weather_agent, query)
    print(tool_calls)

    assert len(tool_calls) == 1
    assert tool_calls[0]["function"]["name"] == "get_weather"

test_calls_weather_when_asked("What's the weather in NYC?")

[{'id': 'call_nSdRjPIAKKP2ql66E5Zollhd', 'function': {'arguments': '{"location":"New York City"}', 'name': 'get_weather'}, 'type': 'function'}]


In [9]:
def test_does_not_call_weather_when_not_asked(query):
    tool_calls = run_and_get_tool_calls(weather_agent, query)
    print(tool_calls)

    assert not tool_calls
test_does_not_call_weather_when_not_asked("Hey")

None


# Triage

In [10]:
from swarm import Agent


def process_refund(item_id, reason="NOT SPECIFIED"):
    """Refund an item. Refund an item. Make sure you have the item_id of the form item_... Ask for user confirmation before processing the refund."""
    print(f"[mock] Refunding item {item_id} because {reason}...")
    return "Success!"


def apply_discount():
    """Apply a discount to the user's cart."""
    print("[mock] Applying discount...")
    return "Applied discount of 11%"


triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
)
sales_agent = Agent(
    name="Sales Agent",
    instructions="Be super enthusiastic about selling bees.",
)
refunds_agent = Agent(
    name="Refunds Agent",
    instructions="Help the user with a refund. If the reason is that it was too expensive, offer the user a refund code. If they insist, then process the refund.",
    functions=[process_refund, apply_discount],
)


def transfer_back_to_triage():
    """Call this function if a user is asking about a topic that is not handled by the current agent."""
    return triage_agent


def transfer_to_sales():
    return sales_agent


def transfer_to_refunds():
    return refunds_agent

triage_agent.functions = [transfer_to_sales, transfer_to_refunds]
sales_agent.functions.append(transfer_back_to_triage)
refunds_agent.functions.append(transfer_back_to_triage)

In [11]:
user_messages = []
for user_message in ['Hi! I need a refund', "It's too expensive"]:
    print(f"User: {user_message}")
    user_messages.append({"role": "user", "content": user_message})
    response = client.run(
        agent=triage_agent,
        messages=user_messages
    )
    swarm_response = response.messages[-1]["content"]
    print(f"Swarm: {swarm_response}")
    user_messages.append({"role": "assistant", "content": swarm_response})

User: Hi! I need a refund
Swarm: I'm here to help you with your refund request. Could you please let me know the reason for the refund? If it's because the item was too expensive, I can offer you a refund code or we can proceed with the refund process.
User: It's too expensive
Swarm: It seems like the refunds process is best handled by a specialized agent. I'll transfer you to our Refunds Agent right away to ensure you get the assistance you need!


In [12]:
from swarm import Swarm
import json

client = Swarm()

def run_and_get_tool_calls(agent, query):
    message = {"role": "user", "content": query}
    response = client.run(
        agent=agent,
        messages=[message],
        execute_tools=False,
    )
    return response.messages[-1].get("tool_calls")

In [13]:
def test_triage_agent_calls_correct_function(query, function_name):
    tool_calls = run_and_get_tool_calls(triage_agent, query)
    print(tool_calls)

    assert len(tool_calls) == 1
    assert tool_calls[0]["function"]["name"] == function_name

test_triage_agent_calls_correct_function("I want to make a refund.", "transfer_to_refunds")

[{'id': 'call_BNQ2LF55KGkNztudlHmBoPUk', 'function': {'arguments': '{}', 'name': 'transfer_to_refunds'}, 'type': 'function'}]
